**Lab 8**

Name: 
Date: 

**As you work through the code below, respond to all in-line comments and questions.**

# Autoencoder

First, let's again build a simple autoencoder for the MNIST data.  This lab is loosely modeled after [Building Autoencoders in Keras](https://blog.keras.io/building-autoencoders-in-keras.html).

In [ ]:
from tensorflow.keras.datasets import mnist

# Load in the MNIST data
(train_X, train_Y), (test_X, test_Y) = mnist.load_data()

N = int(.8 * train_X.shape[0])
val_X = train_X[N:]/ 255
val_Y = train_Y[N:]
train_X = train_X[:N] / 255
train_Y = train_Y[:N]

from keras.models import Sequential, Model
from keras.layers import Flatten, Dense, Embedding, Dropout, Input, Conv2D, MaxPooling2D, Reshape, Conv2DTranspose, UpSampling2D, GlobalMaxPooling2D
from keras import regularizers

encoder_input = Input(shape=(28, 28, 1), name="original_img")
x = Conv2D(16, 3, activation="relu")(encoder_input)
x = Conv2D(32, 3, activation="relu")(x)
x = MaxPooling2D(3)(x)
x = Conv2D(32, 3, activation="relu")(x)
x = Conv2D(16, 3, activation="relu")(x)
x = GlobalMaxPooling2D()(x)
encoder_output = Dense(10, activation='relu')(x)

encoder = Model(encoder_input, encoder_output, name="encoder")
#encoder.summary()

decoder_input = Input(shape=(10,), name="encoded_img")
x = Dense(16, activation='relu')(decoder_input)
x = Reshape((4, 4, 1))(x)
x = Conv2DTranspose(16, 3, activation="relu")(x)
x = Conv2DTranspose(32, 3, activation="relu")(x)
x = UpSampling2D(3)(x)
x = Conv2DTranspose(16, 3, activation="relu")(x)
decoder_output = Conv2DTranspose(1, 3, activation="relu")(x)

decoder = Model(decoder_input, decoder_output, name="decoder")
#decoder.summary()

autoencoder_input = Input(shape=(28, 28, 1), name="img")
encoded_img = encoder(autoencoder_input)
decoded_img = decoder(encoded_img)
autoencoder = Model(autoencoder_input, decoded_img, name="autoencoder")

autoencoder.compile(optimizer='rmsprop', loss='binary_crossentropy')
autoencoder.fit(train_X, train_X,
                epochs=25,
                batch_size=256,
                shuffle=True,
                validation_data=(val_X, val_X))

Let's look at a few images, their codes and their reconstructions.

In [ ]:
# import matplotlib for visualization
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

import numpy as np
np.random.shuffle(val_X)

sample = val_X[:3,:]

print(sample.shape, type(sample))


plt.imshow(sample[1,:].reshape(28,28),cmap='gray')
plt.show()

x = encoder.predict(sample)
print(x)
reconstructions = autoencoder.predict(sample)

print(reconstructions.shape, type(reconstructions))


plt.imshow(reconstructions[1,:].reshape(28,28),cmap='gray')
plt.show()

# Generate from the Autoencoder

Use the autoencoder to generate some data.  How good is your generated data?  How might you improve the quality of your generated data?